In [10]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
import tensorflow as tf
import cv2
import glob
from PIL import Image
from matplotlib.widgets import Button
from pose_parameter import PoseParameter
import sys
from scipy import signal
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.draw import disk

model = tf.keras.models.load_model('trained_model/ICAIPose/models/ICAIPose_best.h5')


In [61]:
# model.summary()
model(np.ones((1,384,216,3), dtype=np.float32))


[<tf.Tensor: shape=(1, 96, 54, 28), dtype=float32, numpy=
 array([[[[ 3.93864699e-04, -5.39175235e-05, -3.78861761e-04, ...,
           -7.81459967e-05,  4.12051973e-04,  6.26530964e-05],
          [ 2.81855930e-04,  9.09755472e-05, -2.95767037e-04, ...,
           -2.48170109e-04,  2.74888705e-04,  2.89724208e-04],
          [ 2.55384017e-04, -1.16673997e-04, -1.47689658e-04, ...,
            2.23269686e-04,  6.39524253e-04,  3.64409760e-04],
          ...,
          [ 1.75902154e-04, -1.88348349e-04, -1.13270798e-04, ...,
           -4.86115459e-05,  5.20024856e-04,  2.17904802e-04],
          [ 1.09504443e-04, -2.83941394e-04, -6.39771170e-05, ...,
           -2.86050723e-04,  4.68959159e-04,  1.95197761e-04],
          [ 1.10602006e-04,  1.61075033e-04, -1.90700812e-04, ...,
            1.02346810e-03,  8.98333907e-04,  7.71378167e-04]],
 
         [[ 3.04556452e-04, -1.48224877e-04, -3.68283392e-04, ...,
           -4.45891637e-05,  7.66614161e-04,  1.50321051e-04],
          [ 4.

In [11]:
class PoseDraw():
    def __init__(self):
        poseParamObj    = PoseParameter()
        self.limbs      = poseParamObj.getLimbs()
        self.limb_color = poseParamObj.getLimbColor()
        self.part_color = poseParamObj.getPartColor()
        self.parts = poseParamObj.getBodyParts()

    def drawPose(self, inputImg, data):
        data_int = np.int32(data[...,::-1])

        # Draw limbs
        for data_pers in data_int:
            for i, pair in enumerate(self.limbs):
                if np.all(data_pers[pair]):
                    inputImg = cv2.line(inputImg, tuple(data_pers[pair[0]]),
                                        tuple(data_pers[pair[1]]),
                                        self.limb_color[i], 2)

        # Draw body parts
        non_zero_data = data_int[np.all(data_int, axis=-1)]
        idx = np.tile(np.arange(data_int.shape[1])[np.newaxis], (data_int.shape[0],1))[np.all(data_int, axis=-1)]
        for i, data_kp in zip(idx, non_zero_data):
            inputImg = cv2.circle(inputImg, tuple(data_kp),3, self.part_color[i],
                                thickness=-1)

        return inputImg
    
def preprocess_img(img, target_size):
    diff = np.max(img.shape[:-1]) - np.min(img.shape[:-1])
    border_1 = (diff + 1) // 2
    border_2 = diff // 2

    # Crop
    if np.argmax(img.shape[:-1]) == 0:
        img = img[border_1:img.shape[0]-border_2]
    elif np.argmax(img.shape[:-1]) == 1:
        img = img[:,border_1:img.shape[1]-border_2]
    else:
        raise Exception('img has invalid shape!')

    # Scaling
    img = resize(img, (target_size,target_size), anti_aliasing=True)

    # Correct dtype and shape
    if np.max(img) <= 1.0:
        img = np.float32(img * 255.0)[np.newaxis]
    else:
        img = np.float32(img)[np.newaxis]

    return img



In [51]:
def DL(img):
    img_size = 256
   
    # Load model
    # tf.keras.utils.plot_model(model, show_shapes=True)
    # Load validation image
    # img = imread('out.jpg')
    img_pp = preprocess_img(img, img_size)
    offset = np.array([103.939, 116.779, 123.68], dtype='float32')[None,None,None,:]
    print(np.min(img_pp[...,::-1]-offset), np.max(img_pp[...,::-1]-offset), np.mean(img_pp[...,::-1]-offset))

    # Run ICAIPose
    out = model(img_pp[...,::-1]-offset)
    return out

def post_old(conf, img):
    
    drawObj = PoseDraw()
    parts = drawObj.parts
    org_size = np.shape(img)[0]
    conf_thresh = 0.2
    conf_map = np.moveaxis(conf[-1][0,:,:,:-1], [0,1,2], [1,2,0])
    kp_location = np.array([np.unravel_index(np.argmax(cm), cm.shape) for cm in conf_map])
    kp_confidence = np.array([np.max(cm) for cm in conf_map])
    kp_location[kp_confidence < conf_thresh] = 0

    # Draw result
    img_draw = drawObj.drawPose(img, org_size/256*kp_location[None])
    return img_draw

In [41]:
def post_new(conf, img):

    # conf_in = np.copy(conf)

    conf_map = np.moveaxis(conf[-1][0,:,:,:-1], [0,1,2], [1,2,0])
    org_size = np.shape(img)[0]
    conf_map_local = np.copy(conf_map)
    conf_thresh = 0.2
    color_part = [(153,0,51),  # Nose
                (153,0,0),   # Neck
                (153,51,0),  # RShoulder
                (153,102,0), # RElbow
                (153,153,0), # RWrist
                (102,153,0), # LShoulder
                (51,153,0),  # LElbow
                (0,153,0),   # LWrist
                (153,0,0),   # MidHip
                (0,153,51),  # RHip
                (0,153,102), # RKnee
                (0,153,153), # RAnkle
                (0,102,153), # LHip
                (0,51,153),  # LKnee
                (0,0,153)]   # LAnkle

    limbs   = np.array([[0,1],
                        [1,2],
                        [2,3],
                        [3,4],
                        [1,5],
                        [5,6],
                        [6,7],
                        [1,8],
                        [8,9],
                        [9,10],
                        [10,11],
                        [8,12],
                        [12,13],
                        [13,14]], dtype='uint16')

    color_limb =        [(153,0,51),  # Neck
                        (153,51,0),  # RShoulder
                        (153,102,0), # RUpperArm
                        (153,153,0), # RForearm
                        (102,153,0), # LShoulder
                        (51,153,0),  # LUpperArm
                        (0,153,0),   # LForearm
                        (153,0,0),   # Back
                        (0,153,51),  # RHiplimbs
                        (0,153,102), # RTight
                        (0,153,153), # RLowerLeg
                        (0,102,153), # LHip
                        (0,51,153),  # LTight
                        (0,0,153)]   # LLowerLeg

    keypoints = []
    # for cm in conf_map_local:
    #     cm[cm<0.2] = 0
    #     coordinates = peak_local_max(cm, min_distance=20, exclude_border=0)
    #     keypoints.append((org_size/256*coordinates).astype(np.int32)[...,::-1])

    for cm in conf_map_local:
        kp = []
        # cm[cm<0.2] = 0
        while True:
            max = np.unravel_index(np.argmax(cm), cm.shape)
            if cm[max] < conf_thresh:
                break
            rr, cc = disk(max, 25, shape=(256,256))
            cm[rr, cc] = 0
            kp.append(max)
        keypoints.append((org_size/256*np.array(kp)).astype(np.int32)[...,::-1])


    # keypoints = np.int32(org_size/256*np.array(keypoints))[...,::-1]
    cnt = 0
    for kps in keypoints:
        for kp in kps:
            img = cv2.circle(img, tuple(kp), 3, color_part[cnt], thickness=-1)
        cnt += 1


    for k, (a,b) in enumerate(limbs):
        pts = []
        flag = False
        dist_out = []
        for kp in keypoints[a]:
            if len(keypoints[a]) > len(keypoints[b]):
                dist = []
                flag = True
                for j in range(len(keypoints[b])):
                    dist.append(np.linalg.norm(kp-keypoints[b][j]))
                    pts.append([kp, keypoints[b][j]])
                dist_out.append(dist)
            else:
                dist = []
                for j in range(len(keypoints[b])):
                    dist.append(np.linalg.norm(kp-keypoints[b][j]))
                pts.append([kp, keypoints[b][np.argmin(dist)]])
        if flag:
            pts = np.array(pts)
            pts = pts[np.argsort(np.array(dist_out).flatten())[:min(len(keypoints[a]), len(keypoints[b]))]]
            flag = False
        for p1, p2 in pts:
            if np.linalg.norm(p1-p2) < org_size/2:
                img = cv2.line(img, p1, p2, color_limb[k], 2)
    return img

def post_conf(conf):
    
    conf = np.moveaxis(conf[-1][0,:,:,:-1], [0,1,2], [1,2,0])

    conf_map = np.copy(conf)

    # conf_map[conf_map<20] = 0

    conf_map = np.sum(conf_map, axis=0)
    # conf_map = (conf_map-np.min(conf_map))/(np.max(conf_map)-np.min(conf_map))
    return conf_map

In [53]:
# imgs = np.load('validation_data/imgs.npy')
imgs = [imread('data/multi.jpg')]
import time
imgs = np.load('validation_data/imgs.npy')

out = []

for i, im in enumerate(imgs): 
    conf = DL(im)
    # np.save('conf.npy',conf[-1])

    # t = time.time()
    out.append(post_new(conf, im))
    # if i % 10 == 0:
    #     print(i, end=', ')
    # new2 = post_conf(conf)
    # new2 = cv2.cvtColor(new2,cv2.COLOR_GRAY2RGB)
    # new = cv2.resize(new, (512, 512))
    # new2 = cv2.resize(new2, (512, 512))

    # img_draw = np.hstack((new2, new/255 ))
    # _, axis = plt.subplots(1,1)
    # axis.imshow(img_draw)
    # axis.axis("off")


-104.66542 91.05102 -5.1741395
-115.555725 146.3489 -14.71828
-101.744774 137.77927 0.41559038
-108.28159 151.061 48.969715
-122.39455 140.53409 67.12635
-123.63397 149.47513 4.484857
-123.63397 131.62894 -12.224598
-123.63397 148.35849 -15.556186
-122.64869 151.061 4.939923
-123.63397 134.00742 18.827208
-123.63397 150.12674 32.25457
-123.63397 147.6019 59.83247
-123.23987 151.061 -32.06503
-123.63397 151.061 -26.500956
-123.63397 151.061 -42.1297
-123.63397 144.71475 57.405025
-123.51681 151.061 -0.8523057
-119.56218 145.60828 12.501882
-123.343765 150.78418 -6.839209
-96.47047 131.70267 39.530415
-123.47236 151.061 -11.149219
-123.51681 151.061 -31.421835
-123.011024 150.89255 19.304214
-123.51681 151.061 -12.147214
-123.51681 151.061 -19.625036
-123.51681 150.94379 -70.02372
-123.51681 151.061 48.720627
-122.559975 151.061 2.8346379
-123.25266 110.55575 -74.12735
-123.51681 151.061 -12.756859
-123.51681 151.061 -19.85945
-123.47826 151.061 75.216385
-122.26066 149.34589 33.047894
-

KeyboardInterrupt: 

In [36]:
ims = np.load('validation_data/imgs.npy')
ims[:1000].shape
np.save('out_new.npy', out)




In [37]:
ims[0]

array([[[ 86.26117 ,  99.9877  ,  41.261173],
        [ 85.84099 ,  98.86072 ,  42.02317 ],
        [ 84.73906 ,  97.29064 ,  41.626152],
        ...,
        [167.17624 , 151.64438 ,  89.46211 ],
        [167.84581 , 148.13289 ,  86.32483 ],
        [168.29982 , 142.93996 ,  79.474495]],

       [[ 88.15261 , 102.15261 ,  43.152603],
        [ 87.79645 , 101.060394,  44.268555],
        [ 85.729515,  98.747406,  42.693752],
        ...,
        [167.9905  , 155.57631 ,  94.61657 ],
        [172.13306 , 157.72922 ,  99.69644 ],
        [176.45143 , 158.50172 ,  99.46295 ]],

       [[ 89.412125, 103.30756 ,  44.621258],
        [ 88.48311 , 101.669   ,  45.111332],
        [ 85.951775,  98.951775,  43.066723],
        ...,
        [165.28157 , 154.17804 ,  96.69041 ],
        [175.60497 , 164.36005 , 112.009605],
        [181.12746 , 168.44225 , 115.395515]],

       ...,

       [[165.98204 , 173.95215 ,  73.75798 ],
        [170.57985 , 178.70732 ,  79.59283 ],
        [176.88911 , 1